In [16]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Connect to Neo4j
def create_connection():
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    return driver

# Initialize the driver
driver = create_connection()

In [17]:
# Create nodes for employees and departments, ensuring no duplicates for departments
def create_employee(driver, name, department):
    with driver.session() as session:
        # Check if the employee already exists
        result = session.run("MATCH (e:Employee {name: $name}) RETURN e", name=name)
        if result.single():
            print(f"Employee '{name}' already exists.")
        else:
            # Merge ensures that the department node is created only if it doesn't already exist
            session.run("""
                MERGE (d:Department {name: $department})
                CREATE (e:Employee {name: $name})-[:WORKS_IN]->(d)
                """, name=name, department=department)
            print(f"Employee '{name}' added to the department '{department}'.")

# Example usage for creating employees
create_employee(driver, "Alice", "Engineering")
create_employee(driver, "Sai", "Accounts")


Employee 'Alice' added to the department 'Engineering'.
Employee 'Sai' added to the department 'Accounts'.


In [18]:
# Retrieve all employees in a specific department
def get_employees_in_department(driver, department):
    with driver.session() as session:
        result = session.run("""
            MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: $department})
            RETURN e.name AS name
            """, department=department)
        employees = [record["name"] for record in result]
        return employees

# Example usage for retrieving employees
employees = get_employees_in_department(driver, "Engineering")
print("Employees in Engineering:", employees)


Employees in Engineering: ['Alice']


In [19]:
# Update an employee's department
def update_employee_department(driver, employee_name, new_department):
    with driver.session() as session:
        # Check if the employee exists before updating
        result = session.run("MATCH (e:Employee {name: $name}) RETURN e", name=employee_name)
        if result.single():
            session.run("""
                MATCH (e:Employee {name: $name})-[r:WORKS_IN]->(d:Department)
                DELETE r
                MERGE (d2:Department {name: $new_department})
                CREATE (e)-[:WORKS_IN]->(d2)
                """, name=employee_name, new_department=new_department)
            print(f"Employee '{employee_name}' has been moved to the '{new_department}' department.")
        else:
            print(f"Employee '{employee_name}' does not exist.")

# Example usage for updating an employee's department
update_employee_department(driver, "Alice", "Engineering")


Employee 'Alice' has been moved to the 'Engineering' department.


In [15]:
# Remove an employee from the graph
def delete_employee(driver, employee_name):
    with driver.session() as session:
        # Check if the employee exists before deleting
        result = session.run("MATCH (e:Employee {name: $name}) RETURN e", name=employee_name)
        if result.single():
            session.run("MATCH (e:Employee {name: $name}) DETACH DELETE e", name=employee_name)
            print(f"Employee '{employee_name}' has been deleted.")
        else:
            print(f"Employee '{employee_name}' does not exist.")

# Example usage for deleting an employee7
delete_employee(driver, "Alice")


Employee 'Alice' has been deleted.
